In [1]:
import os
import psycopg2
import psycopg2.extensions
import logging
import pandas as pd
import numpy as np

/home/pavel/.local/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

In [3]:
logger.info("Создаём подключёние к Postgres")
params = {
    "host": '/var/run/postgresql',
    "port": '5432',
    "user": 'pavel'
}
conn = psycopg2.connect(**params)

2018-08-27 11:01:49,757 : INFO : Создаём подключёние к Postgres


In [4]:
psycopg2.extensions.register_type(
    psycopg2.extensions.UNICODE,
    conn
)
conn.set_isolation_level(
    psycopg2.extensions.ISOLATION_LEVEL_AUTOCOMMIT
)
cursor = conn.cursor()

In [5]:
user_item_query_config = {
    "MIN_USERS_FOR_ITEM": 10,
    "MIN_ITEMS_FOR_USER": 3,
    "MAX_ITEMS_FOR_USER": 50,
    "MAX_ROW_NUMBER": 100000
}
sql_str = (
        """
          SELECT * FROM sales limit 100000;
        """ % user_item_query_config
)

In [6]:
cursor.execute(sql_str)
ui_data = [a for a in cursor.fetchall()]
conn.commit()
logger.info("Данные по продажам загружены из Postgres")

2018-08-27 11:01:50,892 : INFO : Данные по продажам загружены из Postgres


In [7]:
sales = pd.DataFrame(ui_data)

In [8]:
sales.columns = ['OPER_DATE', 'CODE_OFFICE', 'TP', 'DOC_NUMBER', 'ITEM_ARTICLE', 'QTY', 'AMOUNT_AUTO_DISCOUNT', 'AMOUNT_RUR', 'PAYMENT_FORM', 'KEY_DISCOUNT']

In [9]:
sales.head()

,OPER_DATE,CODE_OFFICE,TP,DOC_NUMBER,ITEM_ARTICLE,QTY,AMOUNT_AUTO_DISCOUNT,AMOUNT_RUR,PAYMENT_FORM,KEY_DISCOUNT
0,2017-07-04 08:06:00,127015,I,1127015015336,PB,1,None,500,Наличная,1.0
1,2017-07-04 08:08:00,723044,I,1723044009845,0970083944,1,None,200,Наличная,1.0
2,2017-07-04 08:07:00,672039,I,1672039003109,0970025152,1,None,300,Наличная,1.0
3,2017-07-04 08:06:00,723025,I,1723025003564,K2,1,None,15000,Наличная,NaN
4,2017-07-04 19:29:00,478216,I,1478216002736,0970096274,1,None,200,Наличная,1.0


In [10]:
user_item_query_config = {
    "MIN_USERS_FOR_ITEM": 10,
    "MIN_ITEMS_FOR_USER": 3,
    "MAX_ITEMS_FOR_USER": 50,
    "MAX_ROW_NUMBER": 100000
}
sql_str = (
        """
          SELECT * FROM items limit 100000;
        """ % user_item_query_config
)

In [11]:
cursor.execute(sql_str)
ui_data = [a for a in cursor.fetchall()]
conn.commit()
logger.info("Данные по артикулам загружены из Postgres")

2018-08-27 11:01:52,170 : INFO : Данные по артикулам загружены из Postgres


In [12]:
items = pd.DataFrame(ui_data)

In [13]:
items.columns = ['ITEM_ARTICLE','ITEM_NAME','ITEM_CATEGORY', 'ITEM_SUBCATEGORY','ITEM_MARK','VIEW_ITEM','MOD_NO_COLOR']

In [14]:
items.head()

,ITEM_ARTICLE,ITEM_NAME,ITEM_CATEGORY,ITEM_SUBCATEGORY,ITEM_MARK,VIEW_ITEM,MOD_NO_COLOR
0,0105001240,Чех.UCHUN7BB беж-син. Untamo,Accessories,Чехол,Untamo,Товары без серий,Чехол UCHUN7BB беж-син. Untamo
1,0105001250,Чех.UCHUN7GL серо-салат.Untamo,Accessories,Чехол,Untamo,Товары без серий,Чехол UCHUN7GL серо-салат.Untamo
2,0105001260,Чех. UACBTABBL черн. Untamo,Accessories,Чехол,Untamo,Товары без серий,Чехол UACBTABBL черн. Untamo
3,0105001270,Чех.CPC_AP_I5_BLACK APEXTO,Accessories,Чехол,Lone,Товары без серий,Чех.CPC_AP_I5_BLACK APEXTO
4,0201000490,Р/т Motorola V100 GSM,Phones,FEATUREPHONES,Motorola,Товары без серий,Motorola V100


In [15]:
user_item_query_config = {
    "MIN_USERS_FOR_ITEM": 10,
    "MIN_ITEMS_FOR_USER": 3,
    "MAX_ITEMS_FOR_USER": 50,
    "MAX_ROW_NUMBER": 100000
}
sql_str = (
        """
          SELECT * FROM shops limit 100000;
        """ % user_item_query_config
)

In [16]:
cursor.execute(sql_str)
ui_data = [a for a in cursor.fetchall()]
conn.commit()
logger.info("Данные по справочнику ТТ из Postgres")

2018-08-27 11:01:53,548 : INFO : Данные по справочнику ТТ из Postgres


In [17]:
shops = pd.DataFrame(ui_data)

In [18]:
shops.columns = ['MARKET_CODE', 'SHOP', 'CODE_POINTS_SALE', 'CODE_OFFICE', 'SHEDULE_NAME', 'TIME_ZONE']

In [19]:
shops.head()

,MARKET_CODE,SHOP,CODE_POINTS_SALE,CODE_OFFICE,SHEDULE_NAME,TIME_ZONE
0,SPB,ЦКЗ (Измайловский) Внешние сотрудники,41816,478161,None,None
1,VIP,"Москва, Грекова д.8",056000,277004,График ежедневно 8:00 - 22:00,GMT+03:00
2,YAK,"Республика Саха (Якутия), п. Хандыга,ул, П. Ал...",7M42000,114042,None,None
3,CHT,672038 Чита г Шилова ул. 100 стр. 2,7K71000,175029,None,None
4,BUR,670033 Улан-Удэ г Жердева ул 104 б,7K69000,432083,None,GMT+08:00


In [20]:
user_item_query_config = {
    "MIN_USERS_FOR_ITEM": 10,
    "MIN_ITEMS_FOR_USER": 3,
    "MAX_ITEMS_FOR_USER": 50,
    "MAX_ROW_NUMBER": 100000
}
sql_str = (
        """
          SELECT * FROM retail_offices limit 100000;
        """ % user_item_query_config
)

In [21]:
cursor.execute(sql_str)
ui_data = [a for a in cursor.fetchall()]
conn.commit()
logger.info("Данные по справочнику ТТ из Postgres")

2018-08-27 11:01:54,721 : INFO : Данные по справочнику ТТ из Postgres


In [22]:
retail_offices =  pd.DataFrame(ui_data)

In [23]:
retail_offices.columns = ['YM', 'CODE_POINTS_SALE', 'REGION', 'BRANCHNAME', 'BRANCH_ADRR', 'STATUS']

In [24]:
retail_offices.head()

,YM,CODE_POINTS_SALE,REGION,BRANCHNAME,BRANCH_ADRR,STATUS
0,1707,006000,Б.Москва,Московский филиал,"г. Москва, ул. 1-я Тверская-Ямская, д.2, стр.1...",Открыт
1,1707,055000,Б.Москва,Московский филиал,"г. Москва, Зеленый пр-т д. 54А, ТЦ Меркурий 1 ...",Открыт
2,1707,056000,Б.Москва,Московский филиал,"г. Москва, ул. Грекова д.8",Открыт
3,1707,057000,Б.Москва,Московский филиал,"г. Москва, ул. Маршала Бирюзова д. 14",Открыт
4,1707,058000,Б.Москва,Московский филиал,г. Орехово-Зуево Привокзальная пл. д. 4,Открыт


Реализация Запроса 9. Выгрузить самые дорогие проданные смартфоны в каждом регионе

In [25]:
df = pd.merge(sales, items, how = 'left', on = 'ITEM_ARTICLE')

In [26]:
df1 = df[df.ITEM_SUBCATEGORY == 'SMARTPHONES']

In [27]:
df2 = df1[df1.TP == 'I']

In [28]:
df3 = pd.merge(df2, shops, how = 'left', on = 'CODE_OFFICE')

In [29]:
df4 = pd.merge(df3, retail_offices, on = 'CODE_POINTS_SALE')

In [30]:
df5 = (df4.assign(rn=df4.sort_values(['AMOUNT_RUR'], ascending=False)
                            .groupby(['REGION'])
                            .cumcount() + 1)
              .query('rn <= 1')
             .sort_values(['AMOUNT_RUR','rn'], ascending=False) 
             )

In [31]:
df6 = df5.loc[:, ['REGION','AMOUNT_RUR']]

In [32]:
df6.head()

,REGION,AMOUNT_RUR
644,Сибирь,25990
1578,Б.Москва,21990
472,Волга,19990
391,Запад,18990
317,ДВ,16990


Реализация Запроса 7. Вывести долю каждой позиции в сумме чека

In [33]:
df = pd.merge(sales, items, how = 'left', on = 'ITEM_ARTICLE')

In [34]:
df2 = df.loc[:, ['OPER_DATE', 'DOC_NUMBER', 'ITEM_ARTICLE', 'AMOUNT_RUR']]

In [35]:
df3 = pd.DataFrame(df2.groupby('DOC_NUMBER')['AMOUNT_RUR'].sum())

In [36]:
df4 = pd.merge(df2, df3, how = 'left', on = 'DOC_NUMBER')

In [37]:
df4.columns = ['OPER_DATE', 'DOC_NUMBER', 'ITEM_ARTICLE', 'AMOUNT_RUR', 'Summ_Chek_RUR']

In [38]:
df4['Share_of_item']= df4.Summ_Chek_RUR/df4.AMOUNT_RUR

In [39]:
df4.head()

,OPER_DATE,DOC_NUMBER,ITEM_ARTICLE,AMOUNT_RUR,Summ_Chek_RUR,Share_of_item
0,2017-07-04 08:06:00,1127015015336,PB,500,500,1
1,2017-07-04 08:08:00,1723044009845,0970083944,200,200,1
2,2017-07-04 08:07:00,1672039003109,0970025152,300,300,1
3,2017-07-04 08:06:00,1723025003564,K2,15000,15000,1
4,2017-07-04 19:29:00,1478216002736,0970096274,200,200,1


Выгрузка данных в .csv

In [40]:
df4.to_csv('agg_date_from_postgres.csv', sep=',', header=True)

Реализация Запроса 3. Выгрузить регионы с количеством точкек более 200.

In [41]:
df = shops[shops.CODE_POINTS_SALE != 'NaN']

In [42]:
df1 = pd.merge(df, retail_offices, how = 'left', on = 'CODE_POINTS_SALE')

In [43]:
df2 = df1.groupby(['REGION']).size()

In [44]:
df3 = pd.DataFrame(df2)

In [45]:
df4 = df3.rename(columns = {0: 'COUNT_TT'})

In [46]:
df5 = df4.sort_values(['COUNT_TT'], ascending = False)

In [47]:
df6 = df5[df5['COUNT_TT']>=200]

In [48]:
df6.head()

,COUNT_TT
REGION,
Б.Москва,281
Запад,218


Реализация Запроса 10. Выгрузить кол-во проданных позиций в каждом филиале

In [49]:
df = pd.merge(sales, items, how='left', on = 'ITEM_ARTICLE')

In [50]:
df1 = pd.merge(df, shops, how = 'left', on = 'CODE_OFFICE')

In [51]:
df2 = pd.merge(df1, retail_offices, on = 'CODE_POINTS_SALE')

In [52]:
df3 = df2.loc[:, ['BRANCHNAME', 'DOC_NUMBER']]

In [53]:
df4 = pd.DataFrame(df3.groupby(['BRANCHNAME']).size().sort_values(0, ascending = False))

In [54]:
df5 = df4.rename(columns = {0: 'Кол-во позиций'})

In [55]:
df5.head()

,Кол-во позиций
BRANCHNAME,
Московский филиал,16247
Санкт-Петербургский филиал,6762
Краснодарский филиал,3339
Саратовский филиал,2478
Нижегородский филиал,2080


Реализация Запроса 2. Выгрузить выручку, объем продаж и среднюю цену смартфона в каждом регионе, отсортировать по средней цене.

In [56]:
df = pd.merge(sales, items, how='left', on = 'ITEM_ARTICLE')

In [57]:
df1 = pd.merge(df, shops, how = 'left', on = 'CODE_OFFICE')

In [58]:
df2 = pd.merge(df1, retail_offices, on = 'CODE_POINTS_SALE')

In [59]:
df3 = df2[df2.ITEM_SUBCATEGORY == 'SMARTPHONES']

In [60]:
df4 = df3[df3.TP == 'I']

In [61]:
df5 = df4.apply(pd.to_numeric, errors='ignore')

In [62]:
df6 = df5.groupby(['REGION']).sum()

In [63]:
df7 = df6.loc[:, ['QTY', 'AMOUNT_RUR']]

In [64]:
df7['AVG_PRICE'] = df7.AMOUNT_RUR/df7.QTY

In [65]:
df8 = df7.sort_values(['AVG_PRICE'], ascending = False)

In [66]:
df8.head()

,QTY,AMOUNT_RUR,AVG_PRICE
REGION,,,
Сибирь,173.0,861570.00,4980.173410
Волга,266.0,1180040.00,4436.240602
ДВ,147.0,626830.00,4264.149660
Урал,35.0,146910.00,4197.428571
Юг и СК,233.0,930476.71,3993.462275


Реализация Запроса 8. Вывести долю каждого бренда в общих продажах смартфонов

In [67]:
df = pd.merge(sales, items, how='left', on = 'ITEM_ARTICLE')

In [68]:
df1 = df[df.ITEM_SUBCATEGORY == 'SMARTPHONES']

In [69]:
df2 = df1[df1.TP == 'I']

In [70]:
df3 = df2.apply(pd.to_numeric, errors='ignore')

In [71]:
df4 = df3.groupby(['ITEM_MARK']).sum()

In [72]:
df4.head()

,OPER_DATE,CODE_OFFICE,DOC_NUMBER,ITEM_ARTICLE,QTY,AMOUNT_AUTO_DISCOUNT,AMOUNT_RUR,KEY_DISCOUNT
ITEM_MARK,,,,,,,,
4Good,3.433212e+20,100180191.0,3.211307e+14,1.488500e+12,261.0,184000.00,706790.00,230.0
Alcatel,2.203876e+20,65306812.0,1.935014e+14,6.468002e+11,187.0,1541.10,842665.38,145.0
Archos,1.049397e+19,2451269.0,9.451269e+12,1.200001e+10,7.0,0.00,21430.00,10.0
Asus,2.338839e+20,66498382.0,2.132987e+14,7.176001e+11,160.0,5833.19,1357896.81,164.0
Huawei,1.589143e+20,49187169.0,1.505874e+14,4.240001e+11,106.0,0.00,710352.00,126.0


In [73]:
df4['Summ'] = df4.AMOUNT_RUR.sum()

In [74]:
df4['Share_of_Brand'] = df4.AMOUNT_RUR/df4.Summ

In [75]:
df5 = df4.loc[:, ['QTY','AMOUNT_RUR', 'Summ', 'Share_of_Brand']].sort_values(['Share_of_Brand'], ascending = False)

In [76]:
df5.head()

,QTY,AMOUNT_RUR,Summ,Share_of_Brand
ITEM_MARK,,,,
Micromax,1264.0,2751516.00,11857348.65,0.232052
Xiaomi,124.0,1388484.00,11857348.65,0.117099
Asus,160.0,1357896.81,11857348.65,0.114519
Alcatel,187.0,842665.38,11857348.65,0.071067
Huawei,106.0,710352.00,11857348.65,0.059908
